<h3>Импорты</h3>

In [2]:
import warnings
import pickle

import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline

Отключениие warnings

In [3]:
warnings.filterwarnings('ignore')

Прочитаю обработанные ранее датасеты - без обработки пропусков, с KNN обработкой и с Iterative обработкой:

In [4]:
df_raw_train=pd.read_csv('../data/train_test/train.csv')
df_raw_test=pd.read_csv('../data/train_test/test.csv')

df_knn_train=pd.read_csv('../data/train_test/train_knn_imputer.csv')
df_knn_test=pd.read_csv('../data/train_test/test_knn_imputer.csv')

df_iter_train=pd.read_csv('../data/train_test/train_iter_imputer.csv')
df_iter_test=pd.read_csv('../data/train_test/test_iter_imputer.csv')

dfs={
    'raw':{'train':df_raw_train,'test':df_raw_test},
    'knn':{'train':df_knn_train,'test':df_knn_test},
    'iter':{'train':df_iter_train,'test':df_iter_test}
    }

In [21]:
def load_object(path:str):
    with open(path, 'rb') as f:
        return pickle.load(f)
    
pipe=load_object('d:\\BankProject1\\pipelines\\knn_preproc8.pkl')

In [24]:
numeric_subset=list(df_raw_train.select_dtypes(['float64','int64']).columns)
categoric_subset=list(df_raw_train.select_dtypes('object').columns)

Определю поднаборы признаков:

In [5]:
client_subset=['age','job','marital','education','default','housing','loan']
cur_campaign_subset=['contact','month','day_of_week','duration','campaign','subscribed']
last_campaign_subset=['pdays','previous','poutcome']
soc_econom_subset=['emp.var.rate','cons.price.idx','cons.conf.idx','euribor3m','nr.employed']

numeric_subset=list(df_raw_train.select_dtypes(['float64','int64']).columns)
categoric_subset=list(df_raw_train.select_dtypes('object').columns)

Определю колонки, которые будут использованы после OneHot кодирования категориальных признаков для наборов с неизвестными значениями и без них, а также пределю длины для соответствующих наборов колонок, для дальнейшего использования при создании новых, предобработанных, датасетов:

In [6]:
onehot_categoric_columns_with_unknown=[]
for cat_col in categoric_subset:
    for val in pd.unique(df_raw_train[cat_col]):
        new_column=f'{cat_col}_{val}'
        onehot_categoric_columns_with_unknown.append(new_column)

onehot_categoric_columns_without_unknown=[]
for cat_col in categoric_subset:
    for val in pd.unique(df_knn_train[cat_col]):
        new_column=f'{cat_col}_{val}'
        onehot_categoric_columns_without_unknown.append(new_column)


raw_columns_lenght=len(df_raw_train.columns)
columns_onehot_lenght_with_unknown=len(numeric_subset+onehot_categoric_columns_with_unknown)
columns_onehot_lenght_without_unknown=len(numeric_subset+onehot_categoric_columns_without_unknown)


Определю все пайплайны для предобработки данных, с помощью корорых буду формировать предобработанные датасеты, из которых на следующем этапе выберу наиболее эффективный.

In [7]:
piplines=[]

std_scaler = StandardScaler()
minmax_scaler=MinMaxScaler()

ohe_encoder = OneHotEncoder(handle_unknown='ignore',sparse=False)
original_encoder=OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1)

num_scalers=[std_scaler,minmax_scaler]
cat_encoders=[ohe_encoder,original_encoder]

counter=1
for scaler in num_scalers:
    for encoder in cat_encoders:
        pipe_num=Pipeline([('scaler',scaler)])
        for cat_scaling in num_scalers:

            pipe_cat=Pipeline([('encoder',encoder),('scaler',cat_scaling)])
            col_transformer = ColumnTransformer([('num_preproc', pipe_num, numeric_subset),('cat_preproc', pipe_cat, categoric_subset)])
           
            pipe_name=f'preproc{counter}'
            pipe=Pipeline([(pipe_name, col_transformer)])
            piplines.append(pipe)

            # вывод конфигурации каждого конвеера
            print(pipe_name)
            print('\t for numeric:', scaler.__class__)
            print('\t for categoric:', encoder.__class__)
            print('\t encoded categoric scaler:', cat_scaling.__class__)

            counter+=1
        


preproc1
	 for numeric: <class 'sklearn.preprocessing._data.StandardScaler'>
	 for categoric: <class 'sklearn.preprocessing._encoders.OneHotEncoder'>
	 encoded categoric scaler: <class 'sklearn.preprocessing._data.StandardScaler'>
preproc2
	 for numeric: <class 'sklearn.preprocessing._data.StandardScaler'>
	 for categoric: <class 'sklearn.preprocessing._encoders.OneHotEncoder'>
	 encoded categoric scaler: <class 'sklearn.preprocessing._data.MinMaxScaler'>
preproc3
	 for numeric: <class 'sklearn.preprocessing._data.StandardScaler'>
	 for categoric: <class 'sklearn.preprocessing._encoders.OrdinalEncoder'>
	 encoded categoric scaler: <class 'sklearn.preprocessing._data.StandardScaler'>
preproc4
	 for numeric: <class 'sklearn.preprocessing._data.StandardScaler'>
	 for categoric: <class 'sklearn.preprocessing._encoders.OrdinalEncoder'>
	 encoded categoric scaler: <class 'sklearn.preprocessing._data.MinMaxScaler'>
preproc5
	 for numeric: <class 'sklearn.preprocessing._data.MinMaxScaler'>
	 f

Преобразую исходные датасеты через сознанные конвееры предобработки и сохраню их в .csv формате.

In [8]:
for key in dfs.keys():
    print(key)
    df_train=dfs[key]['train']
    df_test=dfs[key]['test']

    for pipe in piplines:
        pipe_name=pipe.steps[0][0]
        print('\t',pipe_name)
        prepr_df_train=pd.DataFrame(pipe.fit_transform(df_train))
        prepr_df_test=pd.DataFrame(pipe.transform(df_test))

        with open(f'../pipelines/{key}_{pipe_name}.pkl','wb') as f:
            pickle.dump(pipe,f)

        # передаю названия колонок для разных случаев кодирования признаков
        if  len(prepr_df_train.columns)==columns_onehot_lenght_without_unknown:
            prepr_df_train.columns=numeric_subset+onehot_categoric_columns_without_unknown
            prepr_df_test.columns=numeric_subset+onehot_categoric_columns_without_unknown

        elif len(prepr_df_train.columns)==columns_onehot_lenght_with_unknown:
            prepr_df_train.columns=numeric_subset+onehot_categoric_columns_with_unknown
            prepr_df_test.columns=numeric_subset+onehot_categoric_columns_with_unknown
        else:
            prepr_df_train.columns=numeric_subset+categoric_subset
            prepr_df_test.columns=numeric_subset+categoric_subset

        # записываю датасеты по правилу имени исходник_предобработчик_тип.csv
        print('\t\ttrain')
        prepr_df_train.to_csv(f'../data/preprocessed/{key}_{pipe_name}_train.csv',index=False)
        print('\t\ttest')
        prepr_df_test.to_csv(f'../data/preprocessed/{key}_{pipe_name}_test.csv',index=False)


raw
	 preproc1
		train
		test
	 preproc2
		train
		test
	 preproc3
		train
		test
	 preproc4
		train
		test
	 preproc5
		train
		test
	 preproc6
		train
		test
	 preproc7
		train
		test
	 preproc8
		train
		test
knn
	 preproc1
		train
		test
	 preproc2
		train
		test
	 preproc3
		train
		test
	 preproc4
		train
		test
	 preproc5
		train
		test
	 preproc6
		train
		test
	 preproc7
		train
		test
	 preproc8
		train
		test
iter
	 preproc1
		train
		test
	 preproc2
		train
		test
	 preproc3
		train
		test
	 preproc4
		train
		test
	 preproc5
		train
		test
	 preproc6
		train
		test
	 preproc7
		train
		test
	 preproc8
		train
		test


Можно видеть, что для каждого из трёх наборов данных было выполнено по 8 сценариев предобработки для тренировочного и тестового поднабора. Призведена кодировка и масштабирование категориальных признаков разными методами, также отмасштабированы числовые признаки.

Выведу некоторые датасеты: 

In [18]:
fkey='knn'
skey='preproc7'
tkey='train'

pd.read_csv(f'../data/preprocessed/{fkey}_{skey}_{tkey}.csv')

,age,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,job,marital,education,default,housing,loan,poutcome,subscribed
0,1.694917,-0.568265,-0.039529,-0.186533,0.133307,0.442050,1.006984,0.243595,-0.634165,-0.183837,-0.223478,-1.707922,-0.010201,-1.018237,-0.422273,0.170234,-0.242878
1,1.802037,-0.568265,-0.039529,-0.186533,0.133307,0.442050,1.006984,0.243595,-0.634165,0.944348,-0.223478,-0.258062,-0.010201,-1.018237,-0.422273,0.170234,-0.242878
2,-0.340374,-0.568265,-0.039529,-0.186533,0.133307,0.442050,1.006984,0.243595,-0.634165,0.944348,-0.223478,-0.258062,-0.010201,0.982090,-0.422273,0.170234,-0.242878
3,-0.019012,-0.568265,-0.039529,-0.186533,0.133307,0.442050,1.006984,0.243595,-0.634165,-1.029977,-0.223478,-1.224635,-0.010201,-1.018237,-0.422273,0.170234,-0.242878
4,1.694917,-0.568265,-0.039529,-0.186533,0.133307,0.442050,1.006984,0.243595,-0.634165,0.944348,-0.223478,-0.258062,-0.010201,-1.018237,2.368136,0.170234,-0.242878
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28826,1.052193,-0.246897,-0.039529,5.009546,-3.748481,-1.765851,-2.168654,-4.651549,-3.897100,-0.747930,-0.223478,-0.741349,-0.010201,0.982090,-0.422273,-5.271025,-0.242878
28827,-1.304459,0.395839,-0.039529,5.009546,-3.748481,-1.765851,-2.168654,-4.651549,-3.897100,1.508441,1.466736,-0.258062,-0.010201,0.982090,2.368136,-5.271025,-0.242878
28828,0.088109,-0.246897,-0.039529,-0.186533,-3.748481,-1.765851,-2.168654,-4.651549,-3.897100,-0.747930,-1.913692,-1.707922,-0.010201,-1.018237,2.368136,0.170234,-0.242878
28829,-0.233253,0.074471,-0.039529,5.009546,-3.748481,-1.765851,-2.168654,-4.651549,-3.897100,1.508441,-1.913692,-0.258062,-0.010201,0.982090,2.368136,-5.271025,-0.242878


Датасеты сформированы корректно.

Запишу их в каталог data/preprocessed для дальнейшего исследования (понижение размерности) и использования.

Итого записано тренировочный и тестовый надор для каждого из 8-и конвееров предобработки для каждго из трёх наборов данных, которые отличаются методом заполнения пропусков - всего 48 файлов.